In [226]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return (response.choices[0].message.content)

no_condition  = False
no_non_condition_flow = False
filter_flow = []

default_flow = "Basic flow"


In [227]:
def read_file_into_string(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


In [228]:
def write_to_file(filename, s):
    try:
        with open(filename, 'w') as file:
            file.write(s)
        print("Content has been written to", filename)
    except Exception as e:
        print("An error occurred:", e)

In [229]:
file_path_list = [
"D:\GPT-testing\SpecificationData\Book(Github)\Display genre details.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display the list of authors.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display the list of books.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display the list of genres.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Edit the author.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Edit the book.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Edit the genre.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Remove the author.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Remove the book.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Remove the genre.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Add a new author.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Add a new book.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Add a new genre.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display author details.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display book details.txt"
]


In [230]:
file_path = file_path_list[5]
print(file_path)

D:\GPT-testing\SpecificationData\Book(Github)\Edit the book.txt


In [231]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
usecase = "Use case: "+ read_file_into_string(file_path)
if usecase is not None:
    print(usecase)

Use case: Edit the book
Description
The edit page is designed to edit an existing book record in the database.

Main scenario:
User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page;
Application displays form to enter book data;
User enters book data he/she wants to update and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then the book is updated in the database;
If an error occurs, then the error message is displayed;
If a new book record is successfully updated, then the list of the books with updated records is displayed.

Cancel operation scenario:
User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page;
Application displays form to enter book data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t update in t

In [232]:
SYSTEM_PROMPT_extract_flow ="""
Given use case specification flows.
Extract all the contents of flows into json format.
Return the flows in json format.
{
"Flow name":[ contents of corresponding flow],
}
For example: 
{
  "Main flow:": [
    "Step 1: Learner fills in the username field by a valid username that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding username",
    "Step 3: Learner press \"Login\" button",
    "Step 4: System redirects learner to Home page"
  ],
  "Alternative flow 1: Login by email": [
    "At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding email",
    "Go back to step 3 in the basic flow and continue with the steps from step 3"
  ]
}
"""

In [233]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_extract_flow},
    { "role": "user", "content": usecase}
]

gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)
usecase_flow = json.loads(gpt_response)
flows_name = [key for key in usecase_flow]

{
  "Main scenario:": [
    "User clicks the \"Update\" button (to the right from the book he/she wants to update) on the book list/details page",
    "Application displays form to enter book data",
    "User enters book data he/she wants to update and presses \"Submit\" button",
    "If any data is entered incorrectly, incorrect data messages are displayed",
    "If entered data is valid, then the book is updated in the database",
    "If an error occurs, then the error message is displayed",
    "If a new book record is successfully updated, then the list of the books with updated records is displayed"
  ],
  "Cancel operation scenario:": [
    "User clicks the \"Update\" button (to the right from the book he/she wants to update) on the book list/details page",
    "Application displays form to enter book data",
    "User may start entering the data into the form fields",
    "Before pressing \"Submit\" button user presses “Cancel” button",
    "Data doesn’t update in the database, t

In [234]:
for flow_name in flows_name:
    if 'Basic' in flow_name:
        default_flow = flow_name
    if 'Main' in flow_name:
        default_flow = flow_name

print(default_flow)

Main scenario:


In [235]:
SYSTEM_PROMPT_extract_condition ="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the application. 
For each interactive element:
Identify what type of that element (button, icon, card, tab, multiple tabs,scroll, text field, radio buttons, dropdown menu, sliders, switches, dialog, link, form,rating, filter, menu).
If element is used to input/enter data, then determine all the condition of that element which to make make the input valid and the conditions that would render it invalid base on the description of the use case specification. Do not arbitrarily create additional conditions that not mention in the use case flow. If there is no description. If there is no clear description of the element's condition but only mentions valid, invalid, correct, incorrect, leave "condition": {"valid": "valid", "invalid": "invalid"}
If element is used to select a value, then determine all the value of the element.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": "condition of that element", "value": "value of that element", "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters", "invalid": "below 8 characters or over 30 character"}, "value": [], "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "value": [], "type": "button"}}
{"Menu": {"condition": {"valid": "", "invalid": ""}, "value": ['Dashboard','Homepage','Account'], "type": "menu"}}

"""

#có nên đổi cái code lọc fliter_data không????

In [236]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_extract_condition},
    { "role": "user", "content": str(usecase_flow) }
  ]
gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)
full_elements = json.loads(gpt_response)

{
  "Update button": {
    "condition": {
      "valid": "",
      "invalid": ""
    },
    "value": [],
    "type": "button"
  },
  "Submit button": {
    "condition": {
      "valid": "",
      "invalid": ""
    },
    "value": [],
    "type": "button"
  },
  "Cancel button": {
    "condition": {
      "valid": "",
      "invalid": ""
    },
    "value": [],
    "type": "button"
  },
  "Title": {
    "condition": {
      "valid": "valid",
      "invalid": "invalid"
    },
    "value": [],
    "type": "text field"
  },
  "Author": {
    "condition": {
      "valid": "valid",
      "invalid": "invalid"
    },
    "value": [],
    "type": "text field"
  },
  "Genre": {
    "condition": {
      "valid": "valid",
      "invalid": "invalid"
    },
    "value": [],
    "type": "text field"
  },
  "Description": {
    "condition": {
      "valid": "valid",
      "invalid": "invalid"
    },
    "value": [],
    "type": "text field"
  },
  "Publication date": {
    "condition": {
      "valid"

In [237]:
filtered_data = {key: value for key, value in full_elements.items() if value['type'] 
                 not in ['dialog','button', 'tab', 'card','link','icon','scroll bar','scroll']}

In [238]:
for key, value in filtered_data.items():
    print(key)
    print(value)    

Title
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Author
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Genre
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Description
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Publication date
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Menu
{'condition': {'valid': '', 'invalid': ''}, 'value': ['Books', 'Genres', 'Authors', 'Home', 'API'], 'type': 'menu'}


In [239]:
filter_element_click_only = {key: value for key, value in full_elements.items() 
                             if (value['type'] in ['dialog','button', 'tab', 'card','link','icon','scroll bar','title'] and not value['value'])}

In [240]:
filter_element_select_value = {key: value for key, value in full_elements.items() 
                             if ( value['value'])}

In [241]:
for key, value in filter_element_select_value.items():
    print(key)
    print(value)    

Menu
{'condition': {'valid': '', 'invalid': ''}, 'value': ['Books', 'Genres', 'Authors', 'Home', 'API'], 'type': 'menu'}


In [242]:
filter_element_enter_value = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and value['condition']['valid'] and value['condition']['invalid'] )}


In [243]:
for key, value in filter_element_enter_value.items():
    print(key)
    print(value)    

Title
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Author
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Genre
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Description
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}
Publication date
{'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}


In [244]:
SYSTEM_PROMPT_get_flow_name ="""
Given use case flows of a feature and a list of interaction elements extracted from that use case flows.
For each given interactive element:
Identify the flow through which the element is interacted in the use case flow and the type of interaction has with the element and who make that interact.
Ignore interactive elements not mentioned in the given list.
Return the flows in json format.
The JSON format should follow the following structure:
{"Name of interactive element":[{"flow 1":[{"action":"interaction 1","by":"actor","at sentence":"sentence mention that interaction element"},{"action":"interaction 2","by":"actor","at sentence":"sentence mention that interaction element"}]}]}
Examples of output json format template: 
{"Username":[{"Alternative flow 1: Login by valid username":[{"action":"enter","by":"user","at sentence":"Step 4: user enter valid username that above 8 and below 10 characters"}]},{"Exception flow 1: Login by invalid username":[{"action":"fill","by":"user","at sentence":"Step 4: user enter invalid username below 8 characters"}]}]}
{"Rating filter":[{"Alternative flow 1: User rating order":[{"action":"skip","by":"user","at sentence":"Step 3: user skip the rating filter select"},{"action":"select","by":"user","at sentence":"Step 10: user rate the order by select the number of star"}]}]}
"""


In [245]:
Element = "Element: "
for key,value in filter_element_enter_value.items():
    # print(key)
    # print(value)
    Element += key+ " ,"
for key,value in filter_element_select_value.items():
    # print(key)
    # print(value)
    Element += key+ " ,"
Element = Element[:-2] 
Element += "."
print(Element)

Element: Title ,Author ,Genre ,Description ,Publication date ,Menu.


In [246]:
print(Element + "\nUse case flow:" + str(usecase_flow)  )

Element: Title ,Author ,Genre ,Description ,Publication date ,Menu.
Use case flow:{'Main scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User enters book data he/she wants to update and presses "Submit" button', 'If any data is entered incorrectly, incorrect data messages are displayed', 'If entered data is valid, then the book is updated in the database', 'If an error occurs, then the error message is displayed', 'If a new book record is successfully updated, then the list of the books with updated records is displayed'], 'Cancel operation scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User may start entering the data into the form fields', 'Before pressing "Submit" button user presses “Cancel” button', 'Data doesn’t update in

In [247]:
if(len(filter_element_enter_value) != 0 or len(filter_element_select_value) !=0 ):
  promptExtractCondtition = [
      { "role": "system", "content": SYSTEM_PROMPT_get_flow_name},
      { "role": "user", "content":Element + "\nUse case flow:" + str(usecase_flow) }
    ]
  gpt_response = ask(promptExtractCondtition, client, model)
  print(gpt_response)
  element_actions_flow = json.loads(gpt_response)
  print(element_actions_flow)

{
  "Title": [
    {
      "Main scenario:": [
        {
          "action": "enter",
          "by": "user",
          "at sentence": "User enters book data he/she wants to update and presses \"Submit\" button"
        }
      ]
    },
    {
      "Cancel operation scenario:": [
        {
          "action": "start entering",
          "by": "user",
          "at sentence": "User may start entering the data into the form fields"
        }
      ]
    }
  ],
  "Author": [
    {
      "Main scenario:": [
        {
          "action": "enter",
          "by": "user",
          "at sentence": "User enters book data he/she wants to update and presses \"Submit\" button"
        }
      ]
    },
    {
      "Cancel operation scenario:": [
        {
          "action": "start entering",
          "by": "user",
          "at sentence": "User may start entering the data into the form fields"
        }
      ]
    }
  ],
  "Genre": [
    {
      "Main scenario:": [
        {
          "action": 

In [248]:
non_actions = ['display','show']
non_actors = ['system','System']

In [249]:
if(len(filter_element_enter_value) != 0 or len(filter_element_select_value) !=0  ):

    for flow_lists_log in element_actions_flow.values():
        for flow_list_element in flow_lists_log:
            flows_to_delete = []
            for flow_name, actions in flow_list_element.items():
                filtered_actions = []
                for action in actions:
                    if(default_flow in flow_name and 'At step' in action['at sentence'] ):
                        continue

                    if (action['action'] not in non_actions and action['by'] not in non_actors): 
                        filtered_actions.append(action)

                flow_list_element[flow_name] = filtered_actions
                if not filtered_actions:
                    flows_to_delete.append(flow_name)
            flows_to_delete = list(set(flows_to_delete))
            for flow_name in flows_to_delete:
                del flow_list_element[flow_name]

    # Convert modified data back to JSON
    modified_json_content = json.dumps(element_actions_flow, indent=2)
    print(modified_json_content)
    element_flow = element_actions_flow.copy()

{
  "Title": [
    {
      "Main scenario:": [
        {
          "action": "enter",
          "by": "user",
          "at sentence": "User enters book data he/she wants to update and presses \"Submit\" button"
        }
      ]
    },
    {
      "Cancel operation scenario:": [
        {
          "action": "start entering",
          "by": "user",
          "at sentence": "User may start entering the data into the form fields"
        }
      ]
    }
  ],
  "Author": [
    {
      "Main scenario:": [
        {
          "action": "enter",
          "by": "user",
          "at sentence": "User enters book data he/she wants to update and presses \"Submit\" button"
        }
      ]
    },
    {
      "Cancel operation scenario:": [
        {
          "action": "start entering",
          "by": "user",
          "at sentence": "User may start entering the data into the form fields"
        }
      ]
    }
  ],
  "Genre": [
    {
      "Main scenario:": [
        {
          "action": 

In [250]:
print(filter_element_select_value)
print(filter_element_enter_value)
filter_element_select_value_flow_list = []
filter_element_enter_value_flow_list = []


{'Menu': {'condition': {'valid': '', 'invalid': ''}, 'value': ['Books', 'Genres', 'Authors', 'Home', 'API'], 'type': 'menu'}}
{'Title': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Author': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Genre': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Description': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Publication date': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}}


In [251]:
if(len(filter_element_enter_value) != 0 or len(filter_element_select_value) !=0  ):
    print(element_flow)

{'Title': [{'Main scenario:': [{'action': 'enter', 'by': 'user', 'at sentence': 'User enters book data he/she wants to update and presses "Submit" button'}]}, {'Cancel operation scenario:': [{'action': 'start entering', 'by': 'user', 'at sentence': 'User may start entering the data into the form fields'}]}], 'Author': [{'Main scenario:': [{'action': 'enter', 'by': 'user', 'at sentence': 'User enters book data he/she wants to update and presses "Submit" button'}]}, {'Cancel operation scenario:': [{'action': 'start entering', 'by': 'user', 'at sentence': 'User may start entering the data into the form fields'}]}], 'Genre': [{'Main scenario:': [{'action': 'enter', 'by': 'user', 'at sentence': 'User enters book data he/she wants to update and presses "Submit" button'}]}, {'Cancel operation scenario:': [{'action': 'start entering', 'by': 'user', 'at sentence': 'User may start entering the data into the form fields'}]}], 'Description': [{'Main scenario:': [{'action': 'enter', 'by': 'user', '

In [252]:
if(len(filter_element_enter_value) != 0 or len(filter_element_select_value) !=0  ):
    for key in filter_element_select_value:
        if key in element_flow:
            for obj in element_flow[key]:
                filter_element_select_value_flow_list.extend(obj.keys())

        else:
            print(f"No flow found for '{key}' in element_flow.")
    filter_element_select_value_flow_list = list(set(filter_element_select_value_flow_list))
    print(filter_element_select_value_flow_list)


['Cancel operation scenario:']


In [253]:
if(len(filter_element_enter_value) != 0 or len(filter_element_select_value) !=0  ):
    for key in filter_element_enter_value:
        if key in element_flow:
            for obj in element_flow[key]:
                filter_element_enter_value_flow_list.extend(obj.keys())

        else:
            print(f"No flow found for '{key}' in element_flow.")

    filter_element_enter_value_flow_list = list(set(filter_element_enter_value_flow_list))
    print(filter_element_enter_value_flow_list)

['Cancel operation scenario:', 'Main scenario:']


In [254]:
# flow with input value
click_only_flow_name = flows_name.copy()
print(flows_name)

for key in flows_name:
     for key_check in filter_element_enter_value_flow_list:
        if (key in key_check or key_check in key) and key in click_only_flow_name:
            click_only_flow_name.remove(key)
        
for key in flows_name:
    for key_check in filter_element_select_value_flow_list:
        if (key in key_check or key_check in key) and key in click_only_flow_name:
            click_only_flow_name.remove(key)
print(click_only_flow_name)
# click_only_flow_name = [key for key in flows_name if key not in filter_element_enter_value_flow_list]
# click_only_flow_name = [key for key in click_only_flow_name if key not in filter_element_select_value_flow_list]



['Main scenario:', 'Cancel operation scenario:', 'Book details entered during editing:']
['Book details entered during editing:']


In [255]:
print(flows_name)
print(filter_element_enter_value_flow_list)
print(filter_element_select_value_flow_list)
print(click_only_flow_name)


['Main scenario:', 'Cancel operation scenario:', 'Book details entered during editing:']
['Cancel operation scenario:', 'Main scenario:']
['Cancel operation scenario:']
['Book details entered during editing:']


In [256]:
click_only_flows = {}

for key,value in usecase_flow.items():
    if key in click_only_flow_name:
        click_only_flows[key] = value

print("click_only_flows = " + str(click_only_flows))


click_only_flows = {'Book details entered during editing:': ['Title', 'Author', 'Genre', 'Description', 'Publication date']}


In [257]:
input_flows = {}

for key,value in usecase_flow.items():
    if key in filter_element_enter_value_flow_list:
        input_flows[key] = value

print("input_flows = " + str(input_flows))
print(input_flows.keys())


input_flows = {'Main scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User enters book data he/she wants to update and presses "Submit" button', 'If any data is entered incorrectly, incorrect data messages are displayed', 'If entered data is valid, then the book is updated in the database', 'If an error occurs, then the error message is displayed', 'If a new book record is successfully updated, then the list of the books with updated records is displayed'], 'Cancel operation scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User may start entering the data into the form fields', 'Before pressing "Submit" button user presses “Cancel” button', 'Data doesn’t update in the database, then a list of books records is displayed to the user

In [258]:
select_flows = {}

for key,value in usecase_flow.items():
    if key in filter_element_select_value_flow_list:
        select_flows[key] = value

print("select_flows = " + str(select_flows))

select_flows = {'Cancel operation scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User may start entering the data into the form fields', 'Before pressing "Submit" button user presses “Cancel” button', 'Data doesn’t update in the database, then a list of books records is displayed to the user', 'If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be update in the database, and the corresponding form with updated data will be opened.']}


In [259]:
SYSTEM_PROMPT_gen_no_condition_test_scenarios ="""
Given use case specification flows.
Your task is to generate one test scenario only to test each given flow.
Return the test scenarios generated in json format.
The JSON format should follow the following structure:
{
    "Test Scenarios":[ "name of test scenario"]
  }
For example :
{
  "Test Scenarios": [
    "Verify that users can successfully create a new account with valid credentials."
  ]
}
"""

In [260]:
SYSTEM_PROMPT_gen_test_scenarios_input_element ="""
Given a list of interaction element for input value and their extracted conditions and use case flows that reflect those element.
Your task is to generate a comprehensive set of test scenarios that test all given the conditions of given elements.
The test scenarios generated must have clear conditions descritions and mention conditions is tested but do not need example data. 
The positive test scenarios generated must verify the combination of valid elements (the combination follow the description in the use case flow). 
The negative test scenarios generated must verify only one given invalid condition at a time. If there is no invalid condition, you don't need to write negative test scenario.
For cases where the invalid condition does not have a corresponding output description, predict the possible outcome to test.

Return the test scenarios generated in json format.
The JSON format should follow the following structure:
{
  "Positive Test Scenarios": [{ "positive test scenario description": {"conditions": conditions tested}}],
  "Negative Test Scenarios": [{ "negative test scenario description": {"conditions": conditions tested}}]
  }

An example of a test scenario: 
{
  "Positive Test Scenarios": [{ "Verify that users can successfully create a new account with valid email and valid passowrd.": {"conditions": login with email}}],
  "Negative Test Scenarios": [{"Verify that the system shows a notification for invalid password when the password is missing upper character": {"conditions": password must have at least one upper character }}}]
  }
"""

In [261]:
SYSTEM_PROMPT_gen_test_scenarios_select_element ="""
Given a list of element for selecting value, their value options and use case flows use given element.
For each given element:
Generate only one test scenario for each element value options. 
Generate test scenarios for the condition (valid, invalid) if element mentions.

Do not generate test scenario to test element that not mention in the given element list.

The JSON format should follow the following structure:
{
    "Test Scenarios":[ "description of scenario 1"]
  }
For example :
{
  "Test Scenarios": [
    "Verify that users can successfully create a new account with valid credentials."
  ]
}
"""
# Generate test scenarios to test the selection for each element's value.


In [262]:
all_test_scenario = []

In [263]:

if( len(click_only_flows)!= 0 ):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_no_condition_test_scenarios},
    { "role": "user", "content": str(click_only_flows) }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(gpt_response)
    temp = json.loads(gpt_response)["Test Scenarios"]
    all_test_scenario.extend(temp)

    

{
  "Test Scenarios": [
    "Verify that users can successfully edit a book's details including Title, Author, Genre, Description, and Publication date."
  ]
}


In [264]:
print("Element: "+str(filter_element_enter_value) +"\nUse case:" +str(input_flows) )

Element: {'Title': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Author': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Genre': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Description': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}, 'Publication date': {'condition': {'valid': 'valid', 'invalid': 'invalid'}, 'value': [], 'type': 'text field'}}
Use case:{'Main scenario:': ['User clicks the "Update" button (to the right from the book he/she wants to update) on the book list/details page', 'Application displays form to enter book data', 'User enters book data he/she wants to update and presses "Submit" button', 'If any data is entered incorrectly, incorrect data messages are displayed', 'If entered data is valid, then the book is updated in the database', 'If an error occurs, then the error message is dis

In [265]:

if( len(filter_element_enter_value)!= 0 ):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios_input_element},
    { "role": "user", "content": "Element: "+str(filter_element_enter_value) +"\nUse case:" +str(input_flows)  }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    data = json.loads(gpt_response)

    # Extract only the test scenario names
    all_scenario_names = []
    for scenario_type in data:
        for scenario in data[scenario_type]:
            scenario_desc = list(scenario.keys())[0]
            all_scenario_names.append(scenario_desc)

    print("All Test Scenario Names:")
    print(json.dumps(all_scenario_names, indent=2))
    all_test_scenario.extend(all_scenario_names)
    


All Test Scenario Names:
[
  "Verify that the book record is successfully updated with valid Title, Author, Genre, Description, and Publication date",
  "Verify that the system shows an error message when the Title is invalid",
  "Verify that the system shows an error message when the Author is invalid",
  "Verify that the system shows an error message when the Genre is invalid",
  "Verify that the system shows an error message when the Description is invalid",
  "Verify that the system shows an error message when the Publication date is invalid"
]


In [266]:
if( len(filter_element_select_value)!= 0):
    promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios_select_element},
    { "role": "user", "content": str(select_flows) + "Element: "+str(filter_element_select_value) }
  ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(gpt_response)
    select_testscenario = json.loads(gpt_response)["Test Scenarios"]
    all_test_scenario.extend(select_testscenario)
    

{
  "Test Scenarios": [
    "Verify that selecting 'Books' from the menu after starting but canceling an update operation does not update data in the database and opens the books list.",
    "Verify that selecting 'Genres' from the menu after starting but canceling an update operation does not update data in the database and opens the genres list.",
    "Verify that selecting 'Authors' from the menu after starting but canceling an update operation does not update data in the database and opens the authors list.",
    "Verify that selecting 'Home' from the menu after starting but canceling an update operation does not update data in the database and returns the user to the home page.",
    "Verify that selecting 'API' from the menu after starting but canceling an update operation does not update data in the database and opens the API documentation page."
  ]
}


In [267]:
print(filter_element_select_value)


{'Menu': {'condition': {'valid': '', 'invalid': ''}, 'value': ['Books', 'Genres', 'Authors', 'Home', 'API'], 'type': 'menu'}}


In [268]:
test_scenario_file_path = file_path.replace("SpecificationData", "ResultSet\\4.21 - best version\\Propse 1\\")
print(test_scenario_file_path)

D:\GPT-testing\ResultSet\4.21 - best version\Propse 1\\Book(Github)\Edit the book.txt


In [269]:
for i in (all_test_scenario):
    print(i)

Verify that users can successfully edit a book's details including Title, Author, Genre, Description, and Publication date.
Verify that the book record is successfully updated with valid Title, Author, Genre, Description, and Publication date
Verify that the system shows an error message when the Title is invalid
Verify that the system shows an error message when the Author is invalid
Verify that the system shows an error message when the Genre is invalid
Verify that the system shows an error message when the Description is invalid
Verify that the system shows an error message when the Publication date is invalid
Verify that selecting 'Books' from the menu after starting but canceling an update operation does not update data in the database and opens the books list.
Verify that selecting 'Genres' from the menu after starting but canceling an update operation does not update data in the database and opens the genres list.
Verify that selecting 'Authors' from the menu after starting but 

In [270]:
write_to_file(test_scenario_file_path,"")

Content has been written to D:\GPT-testing\ResultSet\4.21 - best version\Propse 1\\Book(Github)\Edit the book.txt
